In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from scipy import stats

In [2]:
path="/Users/guoyalong/Downloads/injuries_2010-2020.csv"
path2="/Users/guoyalong/Downloads/Team_Records.csv"

In [3]:
#read in Team_records file
Team_records=pd.read_csv(path2)
Team_records.head()

,Season,Lg,Team,W,L,W/L%,Finish,SRS,Pace,Rel_Pace,ORtg,Rel_ORtg,DRtg,Rel_DRtg,Playoffs,Coaches,Top WS
0,2017-18,NBA,Boston Celtics,29,10,0.744,1,4.38,95.4,-1.7,108.0,0.2,102.8,-5.0,NaN,B. Stevens (29-10),K. Irving (5.7)
1,2016-17,NBA,Boston Celtics*,53,29,0.646,1,2.25,96.8,0.4,111.2,2.4,108.4,-0.4,Lost E. Conf. Finals,B. Stevens (53-29),I. Thomas (12.5)
2,2015-16,NBA,Boston Celtics*,48,34,0.585,2,2.84,98.5,2.7,106.8,0.4,103.6,-2.8,Lost E. Conf. 1st Rnd.,B. Stevens (48-34),I. Thomas (9.7)
3,2014-15,NBA,Boston Celtics*,40,42,0.488,2,-0.40,95.8,1.9,104.7,-0.9,104.5,-1.1,Lost E. Conf. 1st Rnd.,B. Stevens (40-42),T. Zeller (6.5)
4,2013-14,NBA,Boston Celtics,25,57,0.305,4,-4.97,93.3,-0.6,102.9,-3.8,107.7,1.0,NaN,B. Stevens (25-57),B. Bass (5.1)


In [4]:
#read in injuries csv file
injuries_2010_2020=pd.read_csv(path)
injuries_2010_2020.head()

,Date,Team,Acquired,Relinquished,Notes
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely)
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely)
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely)


In [62]:
##Clean up injuries data

In [6]:
#drop the acquired column 
new_injuries_df=injuries_2010_2020.loc[:,["Date","Team","Relinquished","Notes"]]
new_injuries_df

,Date,Team,Relinquished,Notes
0,2010-10-03,Bulls,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010-10-06,Pistons,Jonas Jerebko,torn right Achilles tendon (out indefinitely)
2,2010-10-06,Pistons,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010-10-08,Blazers,Jeff Ayres,torn ACL in right knee (out indefinitely)
4,2010-10-08,Nets,Troy Murphy,strained lower back (out indefinitely)
...,...,...,...,...
27100,2020-09-30,Lakers,NaN,activated from IL
27101,2020-10-02,Heat,Bam Adebayo,strained neck (DTD)
27102,2020-10-02,Heat,Goran Dragic,placed on IL with torn plantar fascia in left ...
27103,2020-10-02,Heat,NaN,activated from IL


In [53]:
#get unique number of teams
acquired_df["Team"].nunique()

32

In [71]:
#get names of teams
acquired_df["Team"].unique()

array(['Heat', 'Rockets', 'Cavaliers', 'Celtics', 'Grizzlies', 'Hawks',
       'Kings', 'Raptors', 'Warriors', 'Pacers', 'Spurs', 'Wizards',
       'Clippers', 'Blazers', 'Hornets', 'Nets', 'Timberwolves', 'Knicks',
       'Lakers', 'Magic', '76ers', 'Thunder', 'Mavericks', 'Bobcats',
       'Pistons', 'Suns', 'Nuggets', 'Bucks', 'Bulls', 'Jazz', 'Pelicans',
       'Bullets'], dtype=object)

In [8]:
#drop null values for relinquished
cleaned_injuries_df=new_injuries_df.dropna(subset=["Relinquished"])
cleaned_injuries_df

,Date,Team,Relinquished,Notes
0,2010-10-03,Bulls,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010-10-06,Pistons,Jonas Jerebko,torn right Achilles tendon (out indefinitely)
2,2010-10-06,Pistons,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010-10-08,Blazers,Jeff Ayres,torn ACL in right knee (out indefinitely)
4,2010-10-08,Nets,Troy Murphy,strained lower back (out indefinitely)
...,...,...,...,...
27097,2020-09-22,Celtics,Romeo Langford,surgery on right wrist (out for season)
27098,2020-09-23,Heat,Gabe Vincent,sore right knee (DTD)
27099,2020-09-30,Heat,Bam Adebayo,strained left shoulder (DTD)
27101,2020-10-02,Heat,Bam Adebayo,strained neck (DTD)


In [9]:
#get unique number of players missing a game or placed on inactive list
new_injuries_df["Relinquished"].nunique()

1156

In [34]:
duplicate_names=new_injuries_df.duplicated(subset='Relinquished')
duplicate_name = new_injuries_df.loc[duplicate_names, 'Relinquished']
duplicate_name

5          Jonas Jerebko
6          Terrico White
25           Mike Miller
35       Dominic McGuire
42         Carlos Boozer
              ...       
27100                NaN
27101        Bam Adebayo
27102       Goran Dragic
27103                NaN
27104                NaN
Name: Relinquished, Length: 25948, dtype: object

In [38]:
duplicate_name.dropna()

5          Jonas Jerebko
6          Terrico White
25           Mike Miller
35       Dominic McGuire
42         Carlos Boozer
              ...       
27097     Romeo Langford
27098       Gabe Vincent
27099        Bam Adebayo
27101        Bam Adebayo
27102       Goran Dragic
Name: Relinquished, Length: 16404, dtype: object

In [11]:
new_injuries_df["Relinquished"].unique()

array(['Carlos Boozer', 'Jonas Jerebko', 'Terrico White', ...,
       'Chris Silva', 'Vic Law', 'Wenyen Gabriel'], dtype=object)

In [ ]:
###Clean up Team_records Data

In [13]:
clean_team_records=Team_records.loc[:,["Season","Team","W","L","W/L%","Finish"]]
clean_team_records

,Season,Team,W,L,W/L%,Finish
0,2017-18,Boston Celtics,29,10,0.744,1
1,2016-17,Boston Celtics*,53,29,0.646,1
2,2015-16,Boston Celtics*,48,34,0.585,2
3,2014-15,Boston Celtics*,40,42,0.488,2
4,2013-14,Boston Celtics,25,57,0.305,4
...,...,...,...,...,...,...
1478,1999-00,Vancouver Grizzlies,22,60,0.268,7
1479,1998-99,Vancouver Grizzlies,8,42,0.160,7
1480,1997-98,Vancouver Grizzlies,19,63,0.232,6
1481,1996-97,Vancouver Grizzlies,14,68,0.171,7


In [17]:
#get unique number of teams
clean_team_records["Team"].nunique()

112

In [19]:
#get name of teams
clean_team_records["Team"].unique()

array(['Boston Celtics', 'Boston Celtics*', 'Toronto Raptors',
       'Toronto Raptors*', 'New York Knicks', 'New York Knicks*',
       'Philadelphia 76ers', 'Philadelphia 76ers*', 'Syracuse Nationals*',
       'Brooklyn Nets', 'Brooklyn Nets*', 'New Jersey Nets',
       'New Jersey Nets*', 'New York Nets', 'New York Nets*',
       'New Jersey Americans', 'Minnesota Timberwolves',
       'Minnesota Timberwolves*', 'Oklahoma City Thunder',
       'Oklahoma City Thunder*', 'Seattle SuperSonics',
       'Seattle SuperSonics*', 'Denver Nuggets', 'Denver Nuggets*',
       'Denver Rockets*', 'Portland Trail Blazers',
       'Portland Trail Blazers*', 'Utah Jazz', 'Utah Jazz*',
       'New Orleans Jazz', 'Cleveland Cavaliers', 'Cleveland Cavaliers*',
       'Detroit Pistons', 'Detroit Pistons*', 'Fort Wayne Pistons*',
       'Fort Wayne Pistons', 'Milwaukee Bucks', 'Milwaukee Bucks*',
       'Indiana Pacers', 'Indiana Pacers*', 'Chicago Bulls',
       'Chicago Bulls*', 'Golden State Warriors'

In [21]:
miami_heat_df=clean_team_records.loc[clean_team_records["Team"] =="Miami Heat",:]

,Season,Team,W,L,W/L%,Finish
1148,2017-18,Miami Heat,18,17,0.514,2
1149,2016-17,Miami Heat,41,41,0.500,3
1151,2014-15,Miami Heat,37,45,0.451,3
1158,2007-08,Miami Heat,15,67,0.183,5
1163,2002-03,Miami Heat,25,57,0.305,7
1164,2001-02,Miami Heat,36,46,0.439,6
1171,1994-95,Miami Heat,32,50,0.390,4
1173,1992-93,Miami Heat,36,46,0.439,5
1175,1990-91,Miami Heat,24,58,0.293,6
1176,1989-90,Miami Heat,18,64,0.220,5


In [22]:
clean_team_records.loc[clean_team_records["Team"] =="Houston Rockets",:]

,Season,Team,W,L,W/L%,Finish
1304,2017-18,Houston Rockets,25,9,0.735,1
1310,2011-12,Houston Rockets,34,32,0.515,4
1311,2010-11,Houston Rockets,43,39,0.524,5
1312,2009-10,Houston Rockets,42,40,0.512,3
1316,2005-06,Houston Rockets,34,48,0.415,5
1319,2002-03,Houston Rockets,43,39,0.524,5
1320,2001-02,Houston Rockets,28,54,0.341,5
1321,2000-01,Houston Rockets,45,37,0.549,5
1322,1999-00,Houston Rockets,34,48,0.415,6
1330,1991-92,Houston Rockets,42,40,0.512,3
